# Kili Tutorial: Webhooks

In this tutorial, we will show how to use webhooks to monitor actions in Kili, such as a label creation. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way, but also to dive into the actual process of iteratively developing real applications in Kili.

Additionally:

For an overview of Kili, visit kili-technology.com You can also check out the Kili [documentation](https://cloud.kili-technology.com/docs). Our goal is to export labels that can predict whether an image contains a Porsche or a Tesla.

The tutorial is divided into two parts:

1. Why use webhooks?
2. Using Kili's webhook in Python

## 1. Why use webhooks?

Webhooks allow to react to particular action in Kili's database by triggering a callback whenever an action is completed. For instance, here, every time a label is created in frontend (upper panel), the label can be logged in Python (lower right panel):

![gif](./img/websockets.gif)

## 2. Using Kili's webhook in Python

Kili Playground exposes a method `label_created_or_updated` that allows to listen for all actions on labels:

- creation of a new label
- update of an existing label

First of all, you need to authenticate:

In [ ]:
import os

!pip install kili
from kili.client import Kili


api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = 'https://cloud.kili-technology.com/api/label/v2/graphql'

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

Then you can define a callback that will be triggered each time a label gets created/updated:

In [ ]:
project_id = 'CHANGE_ME_FOR_YOUR_PROJECT_ID'

def callback(id, data):
    print(f'New data: {data}\n')

client = kili.label_created_or_updated(
    project_id=project_id, callback=callback)

The method `label_created_or_updated` returns a client allowing you to manage the webhook. 

For example, you can pause or unpause the webhook : 

In [ ]:
client.pause()
client.unpause()

Webhooks have a timeout of around 30 days. After this period, the hook is automatically killed. If you need to stop it before, you can call `close`:

In [ ]:
client.close()

If you need to reset the hook, you can call `reset_timeout`:

In [ ]:
client.reset_timeout()

Finally, you can get the number of seconds the webhook was up with :

In [ ]:
client.get_lifetime()

## Summary
In this tutorial, we accomplished the following:

We introduced the concept of webhook and we used `label_created_or_updated` to trigger a webhook.

You can also visit the Kili [website](https://kili-technology.com/) or Kili [documentation](https://cloud.kili-technology.com/docs/introduction/introduction-to-kili-technology.html) for more info!